**ids-pdl13-tut.ipynb**: This Jupyter notebook is provided by Joachim Vogt for the *Python Data Lab* of the module *CH-700 Introduction to Data Science* offered in Fall 2023 at Constructor University. Jupyter notebooks and other learning resources are available from a dedicated *module platform*.

# Elements of image processing

This tutorial introduces Python tools for displaying and processing images in the context of planetary remote sensing. Follow the instructions below to learn to

- [ ] understand basic image formats and display options,
- [ ] apply geometry transformations and intensity scaling to images,
- [ ] use local filters for denoising, smoothing, and edge detection.

If you wish to keep track of your progress, you may edit this markdown cell, check a box in the list above after having worked through the respective part of this notebook, and save the file.

Only selected basic elements of image processing are covered. For more information, see the suggestions for further reading that are embedded in this notebook.

## Preparation

Run the following code cell to import standard Python data science libraries. The NumPy module facilitates efficient processing of numerical arrays, and is usually imported as `np`. From the matplotlib library we import the package `pyplot` using the standard abbreviation `plt`. Image processing tools are provided in several libraries such as `scipy.ndimage`, `imagio`, `PIL.Image`, and `PIL.ImageFilter`. The magic command `%matplotlib inline` (IPython shell) allows for inline display of graphics.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import imageio.v2 as imageio
from PIL import Image, ImageFilter
%matplotlib inline

The following data files are expected to reside in the working directory. Identify the files on the module platform and upload them to the same folder as this Jupyter notebook.

- `PIA01141.tif`: grayscale image of a region on Mars containing the geologic _Face on Mars_ formation, photographed by NASA's Viking 1 Orbiter on July 25, 1976. The file is available from [NASA's JPL Photojournal website](https://photojournal.jpl.nasa.gov/catalog/PIA01141).
- `march2014_1920x1200.jpeg`: [color image](https://www.nasa.gov/sites/default/files/thumbnails/image/march2014_1920x1200.jpeg) (Credit: NASA/USGS Landsat; Geoscience Australia) featured on [this NASA website](https://www.nasa.gov/image-feature/color-explosion-beautiful-earth). The source image was taken in May 2013 by the Landsat 8 satellite over Western Australia, and then enhanced. 
- `mgn_volc.gif`: [section of a Magellan radar image](https://nssdc.gsfc.nasa.gov/planetary/mgn_volc.gif) as described on the page [Magellan Mission to Venus](https://nssdc.gsfc.nasa.gov/planetary/magellan.html).
- `mgn_volc.png`: png version of `mgn_volc.gif`.
- `vo1_890a68.tiff`: image  taken by the Viking 1 Orbiter, showing the largest mountain of the solar system, namely, the Olympus Mons volcano on Mars. Available from [this NASA website](https://nssdc.gsfc.nasa.gov/imgcat/html/object_page/vo1_890a68.html).

## Introduction to digital images

This section illustrates the array structure of grayscale and color images.

### Grayscale images

Two-dimensional arrays (matrices) can be displayed as images, e.g., by means of the matplotlib function `imshow()`. In contrast to `contour()` or `contourf()`, `imshow()` interpretes array elements as square pixels, and the aspect ratio is preserved. Furthermore, the pixels on the vertical axis are reversed, with zero at the top. Note that `imshow()` uses the default color map (keyword `cmap`). To enforce grayscale displays, set `cmap` to `plt.cm.gray`, or to the string variable 'gray'.

In [ ]:
x,y = np.meshgrid(np.linspace(-2,2,201),np.linspace(-1,1,201))
f = 100*(1 + np.exp(-x**2)*np.sin(2*np.pi*y))
print('Shape of the array f: ',f.shape)
plt.figure(figsize=(15,10))
plt.subplot(221)
plt.contour(f)
plt.colorbar()
plt.title('$f(x,y)$ displayed using contour')
plt.subplot(222)
plt.contourf(f)
plt.colorbar()
plt.title('$f(x,y)$ displayed using contourf')
plt.subplot(223)
plt.imshow(f)
plt.colorbar()
plt.title("$f(x,y)$ displayed using imshow, default cmap")
plt.subplot(224)
plt.imshow(f,cmap='gray')
plt.colorbar()
plt.title("$f(x,y)$ displayed using imshow, cmap='gray'")
#---plt.savefig('contourf_imshow.png',facecolor=None,bbox_inches='tight')

Compared with image display functions of other Python modules, the matplotlib function `imshow()` does not require the image intensity values to be of dtype `unit8` (value range 0-255), and it also applies some automatic rescaling. In the following, display of the full intensity range will be enforced (`vmin=0,vmax=255`) unless otherwise stated.

### Color images

Color images are stored in several two-dimensional arrays. Each of the arrays (also called channels) represents one color according to a color model, typically RGB (red-green-blue). A four-channel version of the RGB model is RGBA, with the channel A (alpha) quantifying transparency.

Load the [image file](https://www.nasa.gov/sites/default/files/thumbnails/image/march2014_1920x1200.jpeg) (Credit: NASA/USGS Landsat; Geoscience Australia) featured on [this NASA website](https://www.nasa.gov/image-feature/color-explosion-beautiful-earth). The source image was taken in May 2013 by the Landsat 8 satellite over Western Australia, and then enhanced. Convince yourself that the data consist of three two-dimensional arrays.

In [ ]:
arr = imageio.imread('march2014_1920x1200.jpeg')
print('Shape of the array arr: ',arr.shape)

Display the full image, and then the three color channels in grayscale. Note the convention shared by matrices and images: the number of rows (here: 1200, `axis=0`) corresponds to the number of pixels in the vertical dimension, and the number of columns (here: 1920, `axis=1`) to the number of pixels in the horizontal dimension.

In [ ]:
plt.figure(figsize=(13,8))
plt.subplot(2,2,1)
plt.imshow(arr,vmin=0,vmax=255)
colstr = ['(red)','(green)','(blue)']
plt.title('Color image: all channels')
plt.ylabel('Vertical dimension [pixels]')
for k in range(3):
    plt.subplot(2,2,k+2)
    plt.imshow(arr[...,k],cmap='gray',vmin=0,vmax=255)
    plt.title('Color image: channel {} '.format(k)+colstr[k])
    if k>0: plt.xlabel('Horizontal dimension [pixels]')
    if 1==k%2: plt.ylabel('Vertical dimension [pixels]')
#---plt.savefig('color_channels.png',facecolor=None,bbox_inches='tight')

Color images can be converted to grayscale using the function `convert()` from the Pillow library `PIL.Image`.

In [ ]:
img = Image.open('march2014_1920x1200.jpeg').convert('L')
img.save('march2014_1920x1200_grayscale.png')
arr_gray = imageio.imread('march2014_1920x1200_grayscale.png')
plt.figure(figsize=(10,5))
plt.imshow(arr_gray,cmap='gray',vmin=0,vmax=255)
plt.colorbar()
plt.title('Color image converted to grayscale')
plt.xlabel('Horizontal dimension [pixels]')
plt.ylabel('Vertical dimension [pixels]')
print('Shape of the array arr_gray: ',arr_gray.shape)

Grayscale images may be offered in color file formats for compatibility reasons, with all color channels containing the same data (possibly supplemented by the alpha channel of the RGBA model), see e.g. the file `mgn_volc.png` on the module platform converted from [mgn_volc.gif](https://nssdc.gsfc.nasa.gov/planetary/mgn_volc.gif) showing a section of a Magellan radar image as described on the page [Magellan Mission to Venus](https://nssdc.gsfc.nasa.gov/planetary/magellan.html). In the code cell below, the converted image is stored in proper grayscale format for later use.

In [ ]:
arr = imageio.imread('mgn_volc.png')
print('Shape of the array arr: ',arr.shape)
img = Image.open('mgn_volc.png').convert('L')
img.save('mgn_volc_grayscale.png')
arr_gray = imageio.imread('mgn_volc_grayscale.png')
print('Shape of the array arr_gray: ',arr_gray.shape)

An image opened with the module `PIL.Image` can be displayed in a Jupyter notebook using the built-in function `display()`.

In [ ]:
display(img)

### Further reading: Introduction to digital images

To familiarize further with the basic concepts of digital image processing in Python, consult the documentation and web resources, e.g., the [SciPy lecture notes](https://scipy-lectures.org/advanced/image_processing/) and the [documentation of the Pillow module (PIL)](https://pillow.readthedocs.io/en/stable/).

## Image transformations

Here we distinguish between transformations of the image geometry (region extractions or cropping, rotations) and scalings of image intensity.

### Geometry transformations

Cropping an image corresponds to extracting a sub-array.

In [ ]:
arr = imageio.imread('mgn_volc_grayscale.png')
plt.figure(figsize=(12,4))
plt.subplot(121)
plt.imshow(arr,cmap='gray',vmin=0,vmax=255)
plt.colorbar()
plt.title('Full image')
plt.subplot(122)
arr_crop = arr[150:350,200:400]
plt.imshow(arr_crop,cmap='gray',vmin=0,vmax=255)
plt.colorbar()
plt.title('Cropped image')

The cropping operation is implemented also in the `PIL.Image` function `crop()`.

In [ ]:
img = Image.open('mgn_volc_grayscale.png')
img_cropped = img.crop((200,150,400,350))
display(img_cropped)

The `PIL.Image` module contains further functions for geometry transformations, e.g., for flipping or rotating an image.

In [ ]:
img_rot45 = img_cropped.rotate(45)
display(img_rot45)

### Intensity scaling

When intensity values are concentrated in a narrow interval, one may map them onto the full intensity range for enhancing the image contrast. The histogram of the transformed image then shows a broader distribution (histogram equalization).

In the example below, the histogram of the original image shows that intensity values are mainly in the interval $[100,170]$. To display the image with enhanced contrast, the keywords `vmin` and `vmax` of `plt.imshow()` are chosen accordingly .

In [ ]:
arr = imageio.imread('mgn_volc_grayscale.png')
plt.figure(figsize=(15,4))
plt.subplot(131)
plt.imshow(arr,cmap='gray',vmin=0,vmax=255)
plt.colorbar()
plt.title('Original image')
plt.subplot(132)
plt.hist(arr.flatten())
plt.title('Histogram of flattened array')
plt.subplot(133)
plt.imshow(arr,cmap='gray',vmin=100,vmax=170)
plt.colorbar()
plt.title('Contrast enhancement')
#---plt.savefig('contrast_enhancement.png',facecolor=None,bbox_inches='tight')

The intensity values can also be rescaled numerically, and the result then displayed using the full intensity range.

In [ ]:
#.. effective range of intensity values in the original image
oldmin = 100
oldmax = 170
#.. range of intensity values for rescaling
newmin = 0
newmax = 255
#.. rescaling formula requires floating operations
arr_f64 = np.asarray(arr,dtype=np.float64)
arr_hst_f64 = newmin + (arr_f64-oldmin)*(newmax-newmin)/(oldmax-oldmin)
#.. convert back to byte array
arr_hst = np.asarray(arr_hst_f64,dtype=np.int32)
#.. map invalid intensity to 0 (black) and 255 (white)
arr_hst[arr_hst<newmin] = newmin
arr_hst[arr_hst>newmax] = newmax
#.. reproduce the previous plot to check
plt.figure(figsize=(15,4))
plt.subplot(131)
plt.imshow(arr,cmap='gray',vmin=0,vmax=255)
plt.colorbar()
plt.title('Original image')
plt.subplot(132)
plt.hist(arr.flatten())
plt.title('Histogram of flattened array')
plt.subplot(133)
plt.imshow(arr_hst,cmap='gray',vmin=0,vmax=255)
plt.colorbar()
plt.title('Contrast enhancement')

### Further reading: Histogram-based techniques

Consult the [SciPy lecture notes](https://scipy-lectures.org/advanced/image_processing/) to learn about histogram equalization and histogram-based segmentation.

## Local filtering

Smoothing and edge detection are two operations where local filters are applied.

### Smoothing filters

To demonstrate how noise reduction in images can be achieved through local filtering, we consider the file [PIA01141.tif](https://photojournal.jpl.nasa.gov/catalog/PIA01141), 
showing a region on Mars photographed by NASA's Viking 1 Orbiter spacecraft in July 1976 (Credit:NASA/JPL). The image is contaminated by salt-and-peppers noise due to bit errors. We focus on the region containing a surface feature called the _Face on Mars_.

In [ ]:
img_full = Image.open('PIA01141.tif')
arr_full = np.asarray(img_full)
arr = arr_full[80:200,280:400]
plt.imshow(arr,cmap='gray',vmin=0,vmax=255)
plt.colorbar()

The SciPy module `ndimage` offers a range of filters. A Gaussian filter reduces the salt-and-pepper noise but affects strongly also the edges in the image. A median filter is better suited to deal with this kind of noise.

In [ ]:
arr_gf = ndimage.gaussian_filter(arr,3)
arr_mf = ndimage.median_filter(arr,5)
plt.figure(figsize=(13,3))
plt.subplot(131)
plt.imshow(arr,cmap='gray',vmin=0,vmax=255)
plt.colorbar()
plt.title('Original image')
plt.subplot(132)
plt.imshow(arr_gf,cmap='gray',vmin=0,vmax=255)
plt.colorbar()
plt.title('Gaussian filter (size 3)')
plt.subplot(133)
plt.imshow(arr_mf,cmap='gray',vmin=0,vmax=255)
plt.colorbar()
plt.title('Median filter (size 5)')
#---plt.savefig('noise_reduction.png',facecolor=None,bbox_inches='tight')

Smoothing filters are implemented also in the `PIL` module `ImageFilter`.

In [ ]:
img = Image.open('PIA01141.tif').crop((280,80,400,200))
img_mfpil = img.filter(ImageFilter.MedianFilter(5))
arr_mfpil = np.asarray(img_mfpil)
plt.imshow(img_mfpil,cmap='gray',vmin=0,vmax=255)
plt.colorbar()
plt.title('Median filter (size 5)')

### Edge detection

Edge detection and sharpening can be achieved by derivative operators such as the Sobel (first derivative) or Roberts (second derivative) filters.

In the code cell below, we demonstrate edge detection as implemented in the `PIL` module `ImageFilter`. The image file [vo1_890a68.tiff](https://nssdc.gsfc.nasa.gov/imgcat/html/object_page/vo1_890a68.html) was taken by the Viking 1 Orbiter. It shows the largest mountain of the solar system, namely,  the Olympus Mons volcano on Mars.

In [ ]:
img = Image.open('vo1_890a68.tiff')
img_edges = img.filter(ImageFilter.FIND_EDGES)
plt.figure(figsize=(10,4))
plt.subplot(121)
plt.imshow(img,cmap='gray',vmin=0,vmax=255)
plt.colorbar()
plt.title('Olympus Mons')
plt.subplot(122)
arr_edges_inv = 255 - np.asarray(img_edges)
plt.imshow(arr_edges_inv,cmap='gray',vmin=0,vmax=255)
plt.colorbar()
plt.title('Edge detection')
#---plt.savefig('edge_detection.png',facecolor=None,bbox_inches='tight')

### Further reading: Local filtering

To learn more about the application of local filters in image processing, read the [Wikipedia article on Mathematical morphology](https://en.wikipedia.org/wiki/Mathematical_morphology).

---
---